# Determination of new hexapod LUT based on scheduler-based CWFS data
new Pressure transducer was first used on sky in February <br>
Feb 4 was the last CWFS dataset(s).<br>
This was used to build and tag v0.8.3.alpha6 in ts_config_attcs (ataos)

In [ ]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient, resample, rendezvous_dataframes
import lsst.daf.butler as dafButler

%matplotlib inline

In [ ]:
def parse_obs_id(obs_id):
    """Return a data_id dictionary from a ObsID (which comes from the
    oods imageInOODS event)
    The dictionary is formatted for a gen3 butler.
    Parameters
    ----------
    obs_id: `str`
        ObsId (e.g. 'AT_O_20200219_000212')
    Returns
    -------
    data_id: `dict`
        dictionary with newly derived day_obs and seq_num keys to be
        used with a butler
    """
    source, controller, day_obs, seq_num = obs_id.split("_")
    day_obs = int(f"{day_obs[0:4]}{day_obs[4:6]}{day_obs[6:8]}")
    seq_num = int(seq_num)

    data_id = {
        "day_obs": day_obs,
        "seq_num": seq_num,
        "detector": 0,
        "instrument": "LATISS",
    }
    return data_id

In [ ]:
def parse_visit_id(visit_id):
    """Return a data_id dictionary from a visit ID (which is returned from
    the take_image command to the ATCamera)
    The dictionary is formatted for a gen3 butler.
    Parameters
    ----------
    visit_id: `int` or `str`
        Visit id (e.g. '2021032300308')
    Returns
    -------
    data_id: `dict`
        dictionary with newly derived day_obs and seq_num keys to be
        used with a butler
    """
    _visit_id = str(visit_id)
    day_obs = int(f"{_visit_id[0:4]}{_visit_id[4:6]}{_visit_id[6:8]}")
    seq_num = int(_visit_id[9::])

    data_id = {
        "day_obs": day_obs,
        "seq_num": seq_num,
        "detector": 0,
        "instrument": "LATISS",
    }

    return data_id

In [ ]:
# DATAPATH = "/repo/main/"
DATAPATH ='LATISS'
butler = dafButler.Butler(
    DATAPATH, instrument="LATISS", collections="LATISS/raw/all"
)

In [ ]:
#efd_client = EfdClient('summit_efd')
efd_client = EfdClient('ldf_stable_efd') 

### Find data
Query for all the `endReadout` events on the timespan of the night.

In [ ]:
date='2022-04-06'
test=date+'-new_Hex_LUT'

In [ ]:
t1_set = Time("2022-02-05T10:12:00", format='isot', scale='utc') # three months
# t1_set = Time("2022-03-01T10:12:00", format='isot', scale='utc') # two months
# t1_set = Time("2022-04-04T10:12:00", format='isot', scale='utc') # one night
t2_set = Time("2022-04-06T10:35:44", format='isot', scale='utc')

In [ ]:
time = Time(Time.now(), format='isot', scale='tai')
print(f'time is {time}')

In [ ]:
end_readout = await efd_client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", 
                                           ["imageName", "requestedExposureTime", "additionalKeys",
                                            "additionalValues","timestampAcquisitionStart","timestampEndOfReadout"], t1_set, t2_set)

In [ ]:
end_readout

In [ ]:
base0 = await efd_client.select_time_series("lsst.sal.Script.logevent_logMessage", 
                                           ["message","level"], t1_set, t2_set)

In [ ]:
base0['message'][base0['message'].str.contains('2022031600233', regex=False)]

In [ ]:
base1=base0[base0['message'].str.contains('2022031600233', regex=False)]

In [ ]:
base2=base0[base0['message'].str.contains('angle', regex=False)]

In [ ]:
# Join the two dataframes to create a single one
base3=base1.join(base2, lsuffix='_caller', rsuffix='_other')
print(base3)

In [ ]:
# print(base0.to_string())

In [ ]:
# Find the start/end sequences and trim the bad ones
# base = base0[(base0.message.str.find(start_log_msg) != -1) | (base0.message.str.find(finish_log_msg) != -1)]

In [ ]:
# # Drop false starts
# if run == 'initial':
#     time_reg = '2021-07-09 02:41:02.488000+00:00'
#     ind=base.index.get_loc(time_reg, method='nearest')
#     base.drop(base.index[ind])

In [ ]:
end_readout

Now match each entry. For each `i` item with `intra` in the name, there must be an `i+1` with `extra` otherwise it is not a pair. 
A pair also has the same groupID
The image before the pair is an in-focus image.

In [ ]:
intra_images = []
extra_images = []
in_focus_images = []
intra_times = []
extra_times = []
in_focus_times = []
intra_exptimes = []
extra_exptimes = []
in_focus_exptimes = []

df=pd.DataFrame()

i = 0
npairs = 0
nmiss = 0

while i < len(end_readout)-3:
    intra = end_readout['imageName'][i]
    extra = end_readout['imageName'][i+1]
    in_focus_5s = end_readout['imageName'][i+2]
    
    #skip known bad files
    # 198 and 199 find different sources...
    if intra == 'AT_O_20210817_000197' and extra == 'AT_O_20210817_000198':
        i+=2
        continue
    # this is horrible, but looks for sequence as it was taken
    # CWFS frames, then 5s in focus object
    # finds the in-focus image by seeing if an SCIENCE was taken right before the pair
    # using a colon to separate values causes issues because there are colons in the timestamp!
    group_id_cwfs_in=(end_readout.additionalValues[i])[0:25]
    img_type_cwfs_in=(end_readout['additionalValues'][i]).split(':')[0]
    group_id_cwfs_out=(end_readout.additionalValues[i+1])[0:25] 
    img_type_cwfs_out=(end_readout['additionalValues'][i+1]).split(':')[0]
    group_id_5s=(end_readout.additionalValues[i+2])[0:25]
    img_type_5s=(end_readout['additionalValues'][i+2]).split(':')[0]
    infocus_exptime=end_readout['requestedExposureTime'][i+2]  # should always be 5s

    if ((group_id_cwfs_in == group_id_cwfs_out) and 
        (group_id_5s != group_id_cwfs_in) and
        ((img_type_cwfs_in and img_type_cwfs_out) == 'ENGTEST') and
        (img_type_5s == 'OBJECT') and
        (infocus_exptime == 5 )):
        
        # read in metadata and check a few things
        data_id=parse_obs_id(end_readout['imageName'][i+2])
        metadata = butler.get('raw.metadata', dataId=data_id, collections=["LATISS/raw/all"])
        if metadata['GRATING'] != 'empty_1':
            print('skipping')
            continue
        
        print(f"Got a pair: {intra} x {extra}, with in-focus of {in_focus_5s}")
        df_tmp=pd.DataFrame({'intra':end_readout['imageName'][i],
                             'extra':end_readout['imageName'][i+1],
                             'inFocus5s':end_readout['imageName'][i+2],
                             # Need times during cwfs for telescope position
                             'inFocusExpTime5s':end_readout['requestedExposureTime'][i+2],
                             'inFocustimestampEndOfReadout':end_readout['timestampEndOfReadout'][i+2],
                             'intraExtratimestampAcquisitionStart':end_readout['timestampAcquisitionStart'][i],
                             'intraExtratimestampEndOfReadout':end_readout['timestampEndOfReadout'][i+1],
                             'filter': metadata['FILTER'],
                            }, index=[end_readout.index[i]])
        # df=df.append(df_tmp) # deprecated
        df=pd.concat([df,df_tmp])
        i+=2
        npairs+=1
    else:
#         print(f"No Match: {intra} x {extra}")
        nmiss+=1
        i+=1

print(f"Got {npairs} pairs and {nmiss} misses.")

In [ ]:
df

In [ ]:
# create new dataframe with new values of interest and we'll join them post-facto
df_offsets=pd.DataFrame()
# Populate the data structure from the pairs found above
for i in range(len(df.index)):
    
    # Determine time stamps for searching for metadata
    # include ability to correct for TAI if required, but set to zero for the moment

#    t1 = Time(in_focus_times[i], scale='tai') - TimeDelta(in_focus_exptimes[i], format='sec', scale='tai')
#    t2 = Time(extra_times[i], scale='tai') - TimeDelta(2., format='sec', scale='tai')
    
    # want time during CWFS sensing for telescope position
    t1 = Time(df['intraExtratimestampAcquisitionStart'][i], format='unix')
    t2 = Time(df['intraExtratimestampEndOfReadout'][i],format='unix')
    
    azel = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                               ["elevationCalculatedAngle99", "azimuthCalculatedAngle99"], t1, t2)
    
    rotator = await efd_client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                  ["nasmyth2CalculatedAngle99"], t1, t2)

    m1_pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure",
                                                  ["pressure"], t1, t2)
    
    # want time during long exposure for hexapod position (or basically just not the CWFS data)
    t2_hex = Time(df['inFocustimestampEndOfReadout'][i], format='unix')
    t1_hex = t1-TimeDelta(df['inFocusExpTime5s'][i], format='sec') # this is subtraction, so before endOfReadout event above
    hexapod_vals = await efd_client.select_time_series("lsst.sal.ATHexapod.positionStatus", 
                                       ["reportedPosition0", "reportedPosition1", "reportedPosition2",
                                       "reportedPosition3", "reportedPosition4", "reportedPosition5"], t1_hex , t2_hex)

# For offsets we want to find the offsets between the start of the set and the beginning of the in-focus image, but the end works too
#     cmd_offset = await efd_client.select_time_series("lsst.sal.ATAOS.command_offset",
#                                                  ["u", "v", "w", "x", "y", "z"], offset_start , Time(df['inFocustimestampEndOfReadout'][i], format='unix_tai'))
    
    # to use the dataframe.between_time(), convert astropy Time object, to numpy time object, to pandas time object, and get the datetime.time
#     time1=pd.to_datetime(t1_set.to_datetime()).time()
#     time2=pd.to_datetime((Time(df['inFocustimestampEndOfReadout'][i],format='unix_tai')).to_datetime()).time()

    df_tmp=pd.DataFrame({'rot_pos':np.mean(rotator['nasmyth2CalculatedAngle99']),
                     'el':np.mean(azel['elevationCalculatedAngle99']),
                     'az':np.mean(azel['azimuthCalculatedAngle99']),
                     'x':hexapod_vals['reportedPosition0'].median(),
                     'y':hexapod_vals['reportedPosition1'].median(),
                     'z':hexapod_vals['reportedPosition2'].median(),
                     'u':hexapod_vals['reportedPosition3'].median(),
                     'v':hexapod_vals['reportedPosition4'].median(),
                     'w':hexapod_vals['reportedPosition5'].median(),
                     'm1': np.mean(m1_pressure['pressure']),
#                      'hexXoffset': cmd_offset['x'].sum(),
#                      'hexYoffset': cmd_offset['y'].sum(),
#                      'hexUoffset': cmd_offset['u'].sum(),
#                      'hexVoffset': cmd_offset['v'].sum(),
#                      'hexZoffset': cmd_offset['z'].sum(),
                        },
                     index=[df.index[i]])
    df_offsets=pd.concat([df_offsets,df_tmp])
    # df_offsets=df_offsets.append(df_tmp) # deprecated

In [ ]:
# Join the two dataframes to create a single one
df=df.join(df_offsets, lsuffix='_caller', rsuffix='_other')

In [ ]:
df

In [ ]:
# out = rendezvous_dataframes(end_readout, cmd_offset, direction='backward', tolerance=pd.Timedelta(days=1))

In [ ]:
filename="data/"+date+'_'+test+"_metadata.csv"
df.to_csv(filename)

## Get M1 mirror data and fit a plane

In [ ]:
pmd = await efd_client.select_time_series("lsst.sal.PMD.position", ["position0", "position1", "position2", "position3", "position5"], t1_set, t2_set)

In [ ]:
# Read in the file (written using code above)
filename="data/"+date+'_'+test+"_metadata.csv"
df2 = pd.read_csv(filename, index_col=0)
df2.index=pd.to_datetime(df2.index)

In [ ]:
df3=rendezvous_dataframes(df2, pmd)

In [ ]:
position0_offset = (df3.position0[0])
position1_offset = (df3.position1[0])
position2_offset = (df3.position2[0])
position3_offset = (df3.position3[0])
position4_offset = (df3.position5[0])

In [ ]:
from scipy import linalg
arr_len = len(df3.position0)
coeff_arr = np.zeros((arr_len,3))
theta_arr = np.zeros((arr_len))
phi_arr = np.zeros((arr_len))
piston_arr = np.zeros((arr_len))
for i in np.arange(arr_len):

    # X, Y, Z - measured from solidmodel
    set2=np.array((  41.0, 468.0, (df3.position2[i]-position2_offset)))
    set3=np.array(( 384.0,-269.0, (df3.position3[i]-position3_offset)))
    set4=np.array((-425.0,-198.0, (df3.position5[i]-position4_offset)))

    # Vector PQ crossed with Vector PR
    normal = np.cross(set3-set2,set4-set2) # gives a,b,c
    #print(f'normal is {normal}')
#     theta_arr[i] = (np.pi/2 + np.arctan2(normal[2],normal[0])) * 206265 # arcsec
#     phi_arr[i] = (np.pi/2 + np.arctan2(normal[2],normal[1])) * 206265   # arcsec
#     piston_arr[i] = normal[2]
    
#     => a * (x - x0) + b * (y - y0) + c * (z - z0) = 0.
# => a * x - a * x0 + b * y - b * y0 + c * z - c * z0 = 0.
# => a * x + b * y + c * z + (- a * x0 - b * y0 - c * z0) = 0. # D is the last terms
    D= -normal[0]*set2[0] - normal[1]*set2[1] - normal[2]*set2[2]  # Constant in plane equation
    # equation 
    
    phi_from_normal = (np.pi/2+np.arctan2(normal[2], normal[1])) * 206265
    theta_from_normal = (np.pi/2+np.arctan2(normal[2], normal[0])) * 206265
    # find z at the origin to represent piston
    Z_origin = -D/normal[2]
    
    # Measure rotation about the Y-axis (perpendicular to elevation)
    # So this is TIP and should result in motion in azimuth
    # get slope by looking at Y=0, X=400
    x_pt=400; y_pt=0
    #Z_at_x_pt= C[0]*x_pt + C[1]*0.0 + C[2]
    Z_at_x_pt= (-D - normal[0]*x_pt - normal[1]*0.0)/normal[2]
    theta = np.arctan2(Z_at_x_pt-Z_origin, x_pt) * 206265 # arcsec
        
    # Measure rotation about the X-axis (aligned to elevation)
    # this is TILT and should result in motion in elevation
    # get slope by looking at Y=0, X=400    
    x_pt=0; y_pt=400
    Z_at_y_pt= (-D - normal[0]*x_pt - normal[1]*y_pt)/normal[2]
    phi = np.arctan2(Z_at_y_pt-Z_origin, y_pt) * 206265 # arcsec

    theta_arr[i] = theta # arcsec
    phi_arr[i] = phi    # arcsec
    piston_arr[i] = Z_origin
    
#     print(f'theta_from_normal is {theta_from_normal:0.2f}, phi_from_normal is {phi_from_normal:0.2f}')
#     print(f'theta_arr is {theta_arr[i]:0.2f} [arcsec], phi_arr is {phi_arr[i]:0.2f} [arcsec]')
    
#     if i == 1:
#         break

In [ ]:
df3

In [ ]:
df3['m1_tip']=theta_arr
df3['m1_tilt']=phi_arr
df3['m1_piston']=piston_arr

df3['m1_y_pos']=(df3.position0-position0_offset)*np.cos(10*np.pi/180)
df3['m1_x_pos']=(df3.position1-position1_offset)*np.cos(10*np.pi/180)

In [ ]:
#write to CSV file
filename="data/"+date+'_'+test+"_data_with_m1_pos.csv"

In [ ]:
df3.to_csv(filename)

#### Plot the M1 data

In [ ]:
# Read in the file (written using code above)
filename="data/"+date+'_'+test+"_data_with_m1_pos.csv"
df3 = pd.read_csv(filename, index_col=0)
df3.index=pd.to_datetime(df3.index)

In [ ]:
df3.keys()

In [ ]:
fig_height=5
fig_width=15

In [ ]:
df_plot=df3
%matplotlib inline
nwide=1; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')

xvals = df_plot.el
yvals=df_plot.y
fit_z = np.polyfit(np.cos(np.radians(90. - xvals)), yvals, 1)
poly_z = np.poly1d(fit_z)

el_range = np.arange(15,87,1)
fit=poly_z(np.cos(np.radians(90. - el_range)))
row1.plot(el_range, fit, '-', color='blue')

xvals1 = df_plot[df_plot['filter']=='empty_1'].el
yvals1 = df_plot[df_plot['filter']=='empty_1'].y
row1.plot(xvals1, yvals1, '.', color='black')

xvals2 = df_plot[df_plot['filter']=='SDSSi'].el
yvals2 = df_plot[df_plot['filter']=='SDSSi'].y
row1.plot(xvals2, yvals2, '.', color='red')

xvals2 = df_plot[df_plot['filter']=='SDSSg'].el
yvals2 = df_plot[df_plot['filter']=='SDSSg'].y
row1.plot(xvals2, yvals2, '.', color='green')

#row1.plot(xvals,yvals, '.', color='black')
row1.set_ylabel('Hexapod Y-position [mm]')

xvals1 = df_plot[df_plot['filter']=='empty_1'].el
yvals1 = df_plot[df_plot['filter']=='empty_1'].x
row2.plot(xvals1, yvals1, '.', color='black')

xvals2 = df_plot[df_plot['filter']=='SDSSi'].el
yvals2 = df_plot[df_plot['filter']=='SDSSi'].x
row2.plot(xvals2, yvals2, '.', color='red')

xvals2 = df_plot[df_plot['filter']=='SDSSg'].el
yvals2 = df_plot[df_plot['filter']=='SDSSg'].x
row2.plot(xvals2, yvals2, '.', color='green')

# yvals=df_plot.x
# row2.plot(xvals,yvals, '.')
xvals = df_plot.el
yvals=df_plot.x
fit_z = np.polyfit(np.cos(np.radians(90. - xvals)), yvals, 1)
poly_z = np.poly1d(fit_z)
fit=poly_z(np.cos(np.radians(90. - el_range)))
row2.plot(el_range, fit, '-', color='blue')

row2.set_ylabel('Hexapod X-position [mm]')


xvals1 = df_plot[df_plot['filter']=='empty_1'].el
yvals1 = df_plot[df_plot['filter']=='empty_1'].z
row3.plot(xvals1, yvals1, '.', color='black')

xvals2 = df_plot[df_plot['filter']=='SDSSi'].el
yvals2 = df_plot[df_plot['filter']=='SDSSi'].z + 0.018
row3.plot(xvals2, yvals2, '.', color='red')

xvals2 = df_plot[df_plot['filter']=='SDSSg'].el
yvals2 = df_plot[df_plot['filter']=='SDSSg'].z + 0.018 
row3.plot(xvals2, yvals2, '.', color='green')

xvals = df_plot.el
yvals=df_plot.z
fit_z = np.polyfit(np.cos(np.radians(90. - xvals)), yvals, 1)
poly_z = np.poly1d(fit_z)
fit=poly_z(np.cos(np.radians(90. - el_range)))
row3.plot(el_range, fit, '-', color='blue')

# yvals=df_plot.z
# row3.plot(xvals,yvals, '.')
row3.set_ylabel('Hexapod Z-position [mm]')
row1.set_xlabel('Elevation [deg]')


In [ ]:
df_plot=df3
%matplotlib inline
nwide=3; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')

# ------- Now hexapod Y and mirror X, and tip

xvals = df_plot.el
yvals=df_plot.y
fit_z = np.polyfit(np.cos(np.radians(90. - xvals)), yvals, 1)
poly_z = np.poly1d(fit_z)

el_range = np.arange(15,87,1)
fit=poly_z(np.cos(np.radians(90. - el_range)))
row1[1].plot(el_range, fit, '-', color='blue')

xvals1 = df_plot[df_plot['filter']=='empty_1'].el
yvals1 = df_plot[df_plot['filter']=='empty_1'].y
row1[1].plot(xvals1, yvals1, '.', color='black')

xvals2 = df_plot[df_plot['filter']=='SDSSi'].el
yvals2 = df_plot[df_plot['filter']=='SDSSi'].y
row1[1].plot(xvals2, yvals2, '.', color='red')

xvals2 = df_plot[df_plot['filter']=='SDSSg'].el
yvals2 = df_plot[df_plot['filter']=='SDSSg'].y
row1[1].plot(xvals2, yvals2, '.', color='green')
row1[1].set_ylabel('Hexapod Y-position [mm]')

yvals = (df_plot.m1_x_pos) # um
xvals = df_plot.el
row1[0].plot(xvals, yvals, 'o-')
row1[0].set_xlabel('Elevation [deg]')
row1[0].set_ylabel('M1 X-position [mm]')

yvals = (df_plot.m1_tip) # um
row1[2].plot(xvals, yvals, 'o-')
row1[2].set_xlabel('Elevation [deg]')
row1[2].set_ylabel('M1 tip [arcsec]')

# ------- Now hexapod X and mirror Y and tilt

xvals1 = df_plot[df_plot['filter']=='empty_1'].el
yvals1 = df_plot[df_plot['filter']=='empty_1'].x
row2[1].plot(xvals1, yvals1, '.', color='black')

xvals2 = df_plot[df_plot['filter']=='SDSSi'].el
yvals2 = df_plot[df_plot['filter']=='SDSSi'].x
row2[1].plot(xvals2, yvals2, '.', color='red')

xvals2 = df_plot[df_plot['filter']=='SDSSg'].el
yvals2 = df_plot[df_plot['filter']=='SDSSg'].x
row2[1].plot(xvals2, yvals2, '.', color='green')

# yvals=df_plot.x
# row2.plot(xvals,yvals, '.')
xvals = df_plot.el
yvals=df_plot.x
fit_z = np.polyfit(np.cos(np.radians(90. - xvals)), yvals, 1)
poly_z = np.poly1d(fit_z)
fit=poly_z(np.cos(np.radians(90. - el_range)))
row2[1].plot(el_range, fit, '-', color='blue')

row2[1].set_ylabel('Hexapod X-position [mm]')

yvals = (df_plot.m1_y_pos) # um
xvals = df_plot.el
row2[0].plot(xvals, yvals, 'o-')
row2[0].set_xlabel('Elevation [deg]')
row2[0].set_ylabel('M1 Y-position [mm]')

yvals = (df_plot.m1_tilt) # um
row2[2].plot(xvals, yvals, 'o-')
row2[2].set_xlabel('Elevation [deg]')
row2[2].set_ylabel('M1 tilt [arcsec]')

# ------- Now hexapod Z and mirror Piston (z)

xvals1 = df_plot[df_plot['filter']=='empty_1'].el
yvals1 = df_plot[df_plot['filter']=='empty_1'].z
row3[1].plot(xvals1, yvals1, '.', color='black')

xvals2 = df_plot[df_plot['filter']=='SDSSi'].el
yvals2 = df_plot[df_plot['filter']=='SDSSi'].z + 0.018
row3[1].plot(xvals2, yvals2, '.', color='red')

xvals2 = df_plot[df_plot['filter']=='SDSSg'].el
yvals2 = df_plot[df_plot['filter']=='SDSSg'].z + 0.018 
row3[1].plot(xvals2, yvals2, '.', color='green')

xvals = df_plot.el
yvals=df_plot.z
fit_z = np.polyfit(np.cos(np.radians(90. - xvals)), yvals, 1)
poly_z = np.poly1d(fit_z)
fit=poly_z(np.cos(np.radians(90. - el_range)))
row3[1].plot(el_range, fit, '-', color='blue')
row3[1].set_ylabel('Hexapod Z-position [mm]')

# yvals=df_plot.z
# row3.plot(xvals,yvals, '.')
row3[0].set_ylabel('M1 Z-position [mm]')
row1[1].set_xlabel('Elevation [deg]')

yvals = (df_plot.m1_piston) # um
xvals = df_plot.el
row3[0].plot(xvals, yvals, 'o-')
row3[0].set_xlabel('Elevation [deg]')
row1[0].set_xlim([15,90])
row2[0].set_xlim([15,90])
row3[0].set_xlim([15,90])
row1[1].set_xlim([15,90])
row2[1].set_xlim([15,90])
row3[1].set_xlim([15,90])

In [ ]:
df_plot2=df_plot[((df_plot['m1_piston']>-0.02) & (df_plot['m1_tilt']>0.5))]

In [ ]:
df_plot=df_plot2
%matplotlib inline
nwide=3; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')

# ------- Now hexapod Y and mirror X, and tip

xvals = df_plot.el
yvals=df_plot.y
fit_z = np.polyfit(np.cos(np.radians(90. - xvals)), yvals, 1)
poly_z = np.poly1d(fit_z)
print(f'Hexapod Y fit is: {poly_z}')

el_range = np.arange(15,87,1)
fit=poly_z(np.cos(np.radians(90. - el_range)))
row1[1].plot(el_range, fit, '-', color='blue')

#overplot the current fit
poly_z = np.poly1d(2.249)
fit=poly_z(np.cos(np.radians(90. - el_range)))
row1[1].plot(el_range, fit, '.', color='pink')

xvals1 = df_plot[df_plot['filter']=='empty_1'].el
yvals1 = df_plot[df_plot['filter']=='empty_1'].y
row1[1].plot(xvals1, yvals1, '.', color='black')

xvals2 = df_plot[df_plot['filter']=='SDSSi'].el
yvals2 = df_plot[df_plot['filter']=='SDSSi'].y
row1[1].plot(xvals2, yvals2, '.', color='red')

xvals2 = df_plot[df_plot['filter']=='SDSSg'].el
yvals2 = df_plot[df_plot['filter']=='SDSSg'].y
row1[1].plot(xvals2, yvals2, '.', color='green')
row1[1].set_ylabel('Hexapod Y-position [mm]')

yvals = (df_plot.m1_x_pos) # um
xvals = df_plot.el
row1[0].plot(xvals, yvals, 'o-')
row1[0].set_xlabel('Elevation [deg]')
row1[0].set_ylabel('M1 X-position [mm]')

yvals = (df_plot.m1_tip) # um
row1[2].plot(xvals, yvals, 'o-')
row1[2].set_xlabel('Elevation [deg]')
row1[2].set_ylabel('M1 tip [arcsec]')

# ------- Now hexapod X and mirror Y and tilt

xvals1 = df_plot[df_plot['filter']=='empty_1'].el
yvals1 = df_plot[df_plot['filter']=='empty_1'].x
row2[1].plot(xvals1, yvals1, '.', color='black')

xvals2 = df_plot[df_plot['filter']=='SDSSi'].el
yvals2 = df_plot[df_plot['filter']=='SDSSi'].x
row2[1].plot(xvals2, yvals2, '.', color='red')

xvals2 = df_plot[df_plot['filter']=='SDSSg'].el
yvals2 = df_plot[df_plot['filter']=='SDSSg'].x
row2[1].plot(xvals2, yvals2, '.', color='green')

# yvals=df_plot.x
# row2.plot(xvals,yvals, '.')
xvals = df_plot.el
yvals=df_plot.x
fit_z = np.polyfit(np.cos(np.radians(90. - xvals)), yvals, 1)
poly_z = np.poly1d(fit_z)
print(f'Hexapod X fit is: {poly_z}')

fit=poly_z(np.cos(np.radians(90. - el_range)))
row2[1].plot(el_range, fit, '-', color='blue')

#overplot the current fit
poly_z = np.poly1d((3.634, -4.934, -2.379))
fit=poly_z(np.cos(np.radians(90. - el_range)))
row2[1].plot(el_range, fit, '.', color='pink')

row2[1].set_ylabel('Hexapod X-position [mm]')

yvals = (df_plot.m1_y_pos) # um
xvals = df_plot.el
row2[0].plot(xvals, yvals, 'o-')
row2[0].set_xlabel('Elevation [deg]')
row2[0].set_ylabel('M1 Y-position [mm]')

yvals = (df_plot.m1_tilt) # um
row2[2].plot(xvals, yvals, 'o-')
row2[2].set_xlabel('Elevation [deg]')
row2[2].set_ylabel('M1 tilt [arcsec]')

# ------- Now hexapod Z and mirror Piston (z)

xvals1 = df_plot[df_plot['filter']=='empty_1'].el
yvals1 = df_plot[df_plot['filter']=='empty_1'].z
row3[1].plot(xvals1, yvals1, '.', color='black')

xvals2 = df_plot[df_plot['filter']=='SDSSi'].el
yvals2 = df_plot[df_plot['filter']=='SDSSi'].z + 0.018
row3[1].plot(xvals2, yvals2, '.', color='red')

xvals2 = df_plot[df_plot['filter']=='SDSSg'].el
yvals2 = df_plot[df_plot['filter']=='SDSSg'].z + 0.018 
row3[1].plot(xvals2, yvals2, '.', color='green')

xvals = df_plot.el
yvals=df_plot.z
fit_z = np.polyfit(np.cos(np.radians(90. - xvals)), yvals, 1)
poly_z = np.poly1d(fit_z)
print(f'Hexapod Z fit is: {poly_z}')

fit=poly_z(np.cos(np.radians(90. - el_range)))
row3[1].plot(el_range, fit, '-', color='blue')

#overplot the current fit
poly_z = np.poly1d((-0.217, -1.045))
fit=poly_z(np.cos(np.radians(90. - el_range)))
row3[1].plot(el_range, fit, '.', color='pink')

row3[1].set_ylabel('Hexapod Z-position [mm]')

# yvals=df_plot.z
# row3.plot(xvals,yvals, '.')
row3[0].set_ylabel('M1 Z-position [mm]')
row1[1].set_xlabel('Elevation [deg]')

yvals = (df_plot.m1_piston) # um
xvals = df_plot.el
row3[0].plot(xvals, yvals, 'o-')
row3[0].set_xlabel('Elevation [deg]')
row1[0].set_xlim([15,90])
row2[0].set_xlim([15,90])
row3[0].set_xlim([15,90])
row1[1].set_xlim([15,90])
row2[1].set_xlim([15,90])
row3[1].set_xlim([15,90])

In [ ]:
170*0.42

In [ ]:
np.std(df_plot2.y)*170*2